In [1]:
%matplotlib inline
import pandas as pd 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from time import time
import gc
from scipy.sparse import csr_matrix
import scipy.sparse as sp
import cPickle as pickle

In [2]:
with open('stored/train_vect_with_meta.pkl','rb') as fp:
    train_vect = pickle.load(fp)

In [3]:
with open('stored/test_vect_with_meta.pkl','rb') as fp:
    test_vect = pickle.load(fp)

In [4]:
train_vect = sp.csr_matrix(train_vect)
test_vect = sp.csr_matrix(test_vect)

In [5]:
train = pd.read_csv('raw/train.csv')
target = train[['TripType','VisitNumber']].copy()
target = target.drop_duplicates(['VisitNumber'])

target.index = target['VisitNumber']
target = target.drop('VisitNumber',1)
target = target.reset_index(drop=True)

In [9]:
target

,TripType
0,999
1,30
2,26
3,8
4,8
5,35
6,41
7,21
8,6
9,42


In [10]:
def to_libfm(df, y, cl, train=False):
    lines = []
    for i in  range(df.shape[0]): 
        item = df[i]
        result = str(y.values[i])
        nums = ''
        for la in sorted(zip(item.indices,item.data)):
            nums += ' '+str(la[0])+':'+str(la[1])
        result+=nums
        lines.append(result)
    
    name = 'test'
    if train:
        name = 'train'
        
    with open('libfm_files/for_fm_'+name+'_'+str(cl)+'.txt','w') as f:
        f.write('\n'.join(lines))

In [6]:
from sklearn.cross_validation import train_test_split
y = target['TripType']
classes = sorted(set(y))
inc = 0

X_train, X_test, y_train_main, y_test_main = train_test_split(train_vect, y, test_size = 0.35, train_size = 0.65, stratify = y)

for cl in classes:
    print cl
    replace_labels = dict(zip(classes, np.zeros(len(classes), dtype = int)))
    replace_labels[cl] = 1
    y_train = y_train_main.replace(replace_labels)
    y_test = y_test_main.replace(replace_labels)
    
    to_libfm(X_train, y_train, cl, True)
    to_libfm(test_vect, pd.Series(np.zeros(test_vect.shape[0])), cl)
    

    inc+=1

'X_train, X_test, y_train_main, y_test_main = train_test_split(train_vect, y, test_size = 0.35, train_size = 0.65, stratify = y)\n\nfor cl in classes:\n    print cl\n    replace_labels = dict(zip(classes, np.zeros(len(classes), dtype = int)))\n    replace_labels[cl] = 1\n    y_train = y_train_main.replace(replace_labels)\n    y_test = y_test_main.replace(replace_labels)\n    \n    to_libfm(X_train, y_train, cl, True)\n    to_libfm(test_vect, pd.Series(np.zeros(test_vect.shape[0])), cl)\n    \n\n    inc+=1'

In [ ]:
# time to play with console
import subprocess
for cl in classes[26:]:
    print cl
    latent = 30
    iters = 300
    cmd = "./libfm/bin/libFM -task c -train libfm_files/for_fm_train_{cl}.txt -test libfm_files/for_fm_test_{cl}.txt -rlog libfm_files/log.txt -dim ’1,1,{latent_num}’ -iter {iter_num}  -out libfm_files/predictions_{cl}.txt"
    return_code = subprocess.call(cmd.format(cl=cl,
                                            latent_num = latent,
                                            iter_num=iters)
                                  , shell=True)  

In [9]:
result = pd.DataFrame()
for cl in classes:
    
    temp = pd.read_csv('libfm_files/predictions_{cl}.txt'.format(cl=cl), header=None)
    result['TripType_'+str(cl)] = pd.Series(temp[0].values)
    
result

,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,TripType_15,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,8.110130e-03,6.533520e-03,1.281830e-05,1.852730e-04,5.157800e-03,1.637110e-02,1.274930e-03,9.577470e-05,6.918790e-03,5.201800e-03,...,6.344310e-03,1.151480e-02,6.997650e-01,0.020075,5.482570e-05,9.452070e-03,6.480950e-02,2.912410e-04,9.479900e-04,4.494750e-02
1,4.191470e-08,1.611590e-02,2.400240e-02,1.109300e-03,2.063250e-02,1.107190e-01,4.139530e-02,1.315560e-04,4.181230e-02,2.205540e-02,...,2.077380e-02,6.771080e-06,1.927710e-02,0.019050,2.782720e-12,6.312990e-05,1.554920e-05,1.174930e-05,4.645760e-03,3.669210e-01
2,3.707920e-03,0.000000e+00,0.000000e+00,4.704300e-03,6.920960e-06,2.951750e-03,1.010570e-03,2.742630e-03,5.287230e-05,5.072960e-06,...,1.794440e-03,3.750350e-03,7.995670e-06,0.003405,4.468880e-03,3.508400e-03,3.538840e-03,4.061330e-03,3.599390e-03,9.573550e-01
3,3.125100e-03,2.589280e-03,5.613050e-09,8.413730e-03,6.017410e-03,1.710470e-01,9.582190e-01,3.678920e-03,5.581930e-04,3.767030e-03,...,3.155430e-03,5.187830e-03,6.449620e-04,0.003882,6.532050e-03,1.848500e-03,8.643570e-03,4.436860e-02,2.361120e-03,2.354680e-02
4,3.707920e-03,0.000000e+00,0.000000e+00,4.704300e-03,6.920960e-06,2.951750e-03,1.010570e-03,2.742630e-03,5.287230e-05,5.072960e-06,...,1.794440e-03,3.750350e-03,7.995670e-06,0.003405,4.468880e-03,3.508400e-03,3.538840e-03,4.061330e-03,3.599390e-03,9.573550e-01
5,7.270910e-03,2.073170e-09,2.196370e-11,8.059210e-03,6.559110e-01,2.123040e-01,3.817660e-02,3.811320e-03,2.843800e-03,4.942810e-03,...,3.083600e-05,1.222830e-02,1.487870e-03,0.075816,5.259600e-03,4.266830e-03,4.634040e-03,1.026380e-02,4.191080e-05,1.596420e-03
6,3.521950e-03,1.145150e-01,3.333330e-03,5.166130e-04,1.195390e-01,1.652450e-03,6.251570e-03,3.260750e-03,2.301140e-05,4.430140e-02,...,2.487610e-03,0.000000e+00,7.556750e-04,0.854384,5.254410e-02,1.048690e-03,4.004100e-02,1.356570e-02,1.599200e-01,1.229850e-03
7,2.878670e-03,1.591320e-16,3.498870e-15,1.673410e-03,2.895760e-12,5.057270e-03,2.794060e-16,1.314600e-02,1.372020e-03,6.612910e-01,...,1.142540e-03,4.912000e-02,3.904380e-03,0.210473,8.135500e-02,4.794720e-03,6.573740e-03,1.662890e-02,4.763680e-05,8.893660e-04
8,1.193710e-04,1.080640e-02,2.024920e-06,2.367460e-07,1.339840e-02,2.391740e-01,2.012480e-02,1.037850e-02,2.360520e-03,3.693320e-03,...,9.589720e-03,1.039880e-02,1.609070e-04,0.881300,1.741080e-04,3.507200e-04,2.965250e-03,2.538020e-03,3.307000e-03,7.861340e-03
9,6.612460e-02,2.040990e-12,1.500670e-07,3.436510e-04,3.264060e-03,8.553050e-02,4.688350e-06,8.462900e-03,3.783630e-02,1.120340e-04,...,1.931080e-03,1.628240e-02,1.923060e-04,0.134471,2.118800e-03,1.351310e-02,2.256710e-02,3.004860e-02,4.835020e-02,3.441620e-03


In [24]:
def multiclass_log_loss(y_true, y_pred, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    https://www.kaggle.com/wiki/MultiClassLogLoss

    idea from this post:
    http://www.kaggle.com/c/emc-data-science/forums/t/2149/is-anyone-noticing-difference-betwen-validation-and-leaderboard-error/12209#post12209

    Parameters
    ----------
    y_true : array, shape = [n_samples]
    y_pred : array, shape = [n_samples, n_classes]

    Returns
    -------
    loss : float
    """
    predictions = np.clip(y_pred, eps, 1 - eps)

    # normalize row sums to 1
    predictions /= predictions.sum(axis=1)[:, np.newaxis]

    actual = np.zeros(y_pred.shape)
    rows = actual.shape[0]
    actual[np.arange(rows), y_true.astype(int)] = 1
    vsota = np.sum(actual * np.log(predictions))
    return -1.0 / rows * vsota

In [10]:
sample_submission = pd.read_csv('raw/sample_submission.csv')

In [11]:
columns = sample_submission.columns
shape = sample_submission.shape

In [12]:
submission = pd.DataFrame(sample_submission['VisitNumber'])
values = pd.DataFrame(result.values)
submission = pd.concat([submission, values], axis=1)
submission.columns = columns
submission

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,8.110130e-03,6.533520e-03,1.281830e-05,1.852730e-04,5.157800e-03,1.637110e-02,1.274930e-03,9.577470e-05,6.918790e-03,...,6.344310e-03,1.151480e-02,6.997650e-01,0.020075,5.482570e-05,9.452070e-03,6.480950e-02,2.912410e-04,9.479900e-04,4.494750e-02
1,2,4.191470e-08,1.611590e-02,2.400240e-02,1.109300e-03,2.063250e-02,1.107190e-01,4.139530e-02,1.315560e-04,4.181230e-02,...,2.077380e-02,6.771080e-06,1.927710e-02,0.019050,2.782720e-12,6.312990e-05,1.554920e-05,1.174930e-05,4.645760e-03,3.669210e-01
2,3,3.707920e-03,0.000000e+00,0.000000e+00,4.704300e-03,6.920960e-06,2.951750e-03,1.010570e-03,2.742630e-03,5.287230e-05,...,1.794440e-03,3.750350e-03,7.995670e-06,0.003405,4.468880e-03,3.508400e-03,3.538840e-03,4.061330e-03,3.599390e-03,9.573550e-01
3,4,3.125100e-03,2.589280e-03,5.613050e-09,8.413730e-03,6.017410e-03,1.710470e-01,9.582190e-01,3.678920e-03,5.581930e-04,...,3.155430e-03,5.187830e-03,6.449620e-04,0.003882,6.532050e-03,1.848500e-03,8.643570e-03,4.436860e-02,2.361120e-03,2.354680e-02
4,6,3.707920e-03,0.000000e+00,0.000000e+00,4.704300e-03,6.920960e-06,2.951750e-03,1.010570e-03,2.742630e-03,5.287230e-05,...,1.794440e-03,3.750350e-03,7.995670e-06,0.003405,4.468880e-03,3.508400e-03,3.538840e-03,4.061330e-03,3.599390e-03,9.573550e-01
5,13,7.270910e-03,2.073170e-09,2.196370e-11,8.059210e-03,6.559110e-01,2.123040e-01,3.817660e-02,3.811320e-03,2.843800e-03,...,3.083600e-05,1.222830e-02,1.487870e-03,0.075816,5.259600e-03,4.266830e-03,4.634040e-03,1.026380e-02,4.191080e-05,1.596420e-03
6,14,3.521950e-03,1.145150e-01,3.333330e-03,5.166130e-04,1.195390e-01,1.652450e-03,6.251570e-03,3.260750e-03,2.301140e-05,...,2.487610e-03,0.000000e+00,7.556750e-04,0.854384,5.254410e-02,1.048690e-03,4.004100e-02,1.356570e-02,1.599200e-01,1.229850e-03
7,16,2.878670e-03,1.591320e-16,3.498870e-15,1.673410e-03,2.895760e-12,5.057270e-03,2.794060e-16,1.314600e-02,1.372020e-03,...,1.142540e-03,4.912000e-02,3.904380e-03,0.210473,8.135500e-02,4.794720e-03,6.573740e-03,1.662890e-02,4.763680e-05,8.893660e-04
8,18,1.193710e-04,1.080640e-02,2.024920e-06,2.367460e-07,1.339840e-02,2.391740e-01,2.012480e-02,1.037850e-02,2.360520e-03,...,9.589720e-03,1.039880e-02,1.609070e-04,0.881300,1.741080e-04,3.507200e-04,2.965250e-03,2.538020e-03,3.307000e-03,7.861340e-03
9,21,6.612460e-02,2.040990e-12,1.500670e-07,3.436510e-04,3.264060e-03,8.553050e-02,4.688350e-06,8.462900e-03,3.783630e-02,...,1.931080e-03,1.628240e-02,1.923060e-04,0.134471,2.118800e-03,1.351310e-02,2.256710e-02,3.004860e-02,4.835020e-02,3.441620e-03


In [13]:
submission.to_csv('submissions/10_fm_30_300.csv',index=False)